In [1]:
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn import linear_model
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
%matplotlib inline

# Challenge: Airline Arrival
## Thinkful Unit 3, Lesson 6


Use this dataset of airline arrival information to predict how late flights will be. A flight only counts as late if it is more than 30 minutes late.

http://stat-computing.org/dataexpo/2009/the-data.html

In [2]:
df = pd.read_csv('datasets/2008.csv')

In [3]:
df = df.drop(['Year'], 1)

In [4]:
df.head()

,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,ActualElapsedTime,CRSElapsedTime,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,1,3,4,2003.0,1955,2211.0,2225,WN,128.0,150.0,...,4.0,8.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
1,1,3,4,754.0,735,1002.0,1000,WN,128.0,145.0,...,5.0,10.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
2,1,3,4,628.0,620,804.0,750,WN,96.0,90.0,...,3.0,17.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
3,1,3,4,926.0,930,1054.0,1100,WN,88.0,90.0,...,3.0,7.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
4,1,3,4,1829.0,1755,1959.0,1925,WN,90.0,90.0,...,3.0,10.0,0,NaN,0,2.0,0.0,0.0,0.0,32.0


In [5]:
# Count nulls 
null_count = df.isnull().sum()
null_count[null_count>0]

DepTime               136246
ArrTime               151649
ActualElapsedTime     154699
CRSElapsedTime           844
AirTime               154699
ArrDelay              154699
DepDelay              136246
TaxiIn                151649
TaxiOut               137058
CancellationCode     6872294
CarrierDelay         5484993
WeatherDelay         5484993
NASDelay             5484993
SecurityDelay        5484993
LateAircraftDelay    5484993
dtype: int64

In [6]:
def add_delay(row):
    return row['CarrierDelay'] + row['WeatherDelay'] + row['NASDelay'] + row['SecurityDelay'] + row['LateAircraftDelay']

df['total_delay'] = df.apply(lambda row: add_delay(row), axis=1)

In [7]:
df.fillna(value={'total_delay': 0})

,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,ActualElapsedTime,CRSElapsedTime,...,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,total_delay
0,1,3,4,2003.0,1955,2211.0,2225,WN,128.0,150.0,...,8.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN,0.0
1,1,3,4,754.0,735,1002.0,1000,WN,128.0,145.0,...,10.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN,0.0
2,1,3,4,628.0,620,804.0,750,WN,96.0,90.0,...,17.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN,0.0
3,1,3,4,926.0,930,1054.0,1100,WN,88.0,90.0,...,7.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN,0.0
4,1,3,4,1829.0,1755,1959.0,1925,WN,90.0,90.0,...,10.0,0,NaN,0,2.0,0.0,0.0,0.0,32.0,34.0
5,1,3,4,1940.0,1915,2121.0,2110,WN,101.0,115.0,...,10.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN,0.0
6,1,3,4,1937.0,1830,2037.0,1940,WN,240.0,250.0,...,7.0,0,NaN,0,10.0,0.0,0.0,0.0,47.0,57.0
7,1,3,4,1039.0,1040,1132.0,1150,WN,233.0,250.0,...,7.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN,0.0
8,1,3,4,617.0,615,652.0,650,WN,95.0,95.0,...,19.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN,0.0
9,1,3,4,1620.0,1620,1639.0,1655,WN,79.0,95.0,...,6.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN,0.0


In [10]:
features = ['Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'CRSDepTime',
       'ArrTime', 'CRSArrTime', 'UniqueCarrier',
       'ActualElapsedTime', 'CRSElapsedTime', 'AirTime', 'ArrDelay',
       'DepDelay', 'Origin', 'Dest', 'Distance', 'TaxiIn', 'TaxiOut',
       'Cancelled', 'Diverted']
X = df[features]
X = pd.get_dummies(X)
Y = df['total_delay']

KeyError: MemoryError()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=20)

In [ ]:
lr = LogisticRegression(C=.9)
fit = lr.fit(X_train, Y_train)
print(lr.score(X_test, Y_test))

In [ ]:
X.dtypes